In [ ]:
import logging
from kiteconnect import KiteConnect
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import time
from IPython.display import clear_output

In [ ]:
logging.basicConfig(level=logging.DEBUG)

api_key = ''
api_secret = ''

kite = KiteConnect(api_key=api_key)

print(kite.login_url())
t = time.localtime()  

In [ ]:
requestToken = ''
data = kite.generate_session(request_token=requestToken, api_secret=api_secret)
kite.set_access_token(data["access_token"])
instruments = kite.instruments()

In [ ]:
def balance():
    funds = kite.margins(segment='equity')
    return funds['available']['live_balance']

In [ ]:
stock_list = [
    'BSE:SHARE_NAME', 
    'NSE:SHARE_NAME', 
]
shareNameList = [
    'SHARE_NAME',
    'SHARE_NAME',
]

In [ ]:
def r(num):
    return round(num, 2)

In [ ]:
def toBuy():
    n = len(stock_list)
    bal = balance()
    

In [ ]:
def placeOrder(shareName, quantity, buy, pricePerShare):
    quantity = 10
    pricePerShare = r(pricePerShare)
    print('price form place order',pricePerShare)
    if buy:
        transactionType = kite.TRANSACTION_TYPE_BUY
        print('BUYING')
    else:
        transactionType = kite.TRANSACTION_TYPE_SELL
        print('SELLING')
    try:
        order_id = kite.place_order(
            variety=kite.VARIETY_REGULAR,
            exchange=kite.EXCHANGE_BSE,
            tradingsymbol=shareName,
            transaction_type=transactionType,
            quantity=quantity,  # quantity
            product=kite.PRODUCT_CNC,
            order_type=kite.ORDER_TYPE_LIMIT,
            price=pricePerShare,
            validity=kite.VALIDITY_DAY
        )
        logging.info("Order placed. ID is: {}".format(order_id))
        return True
    except Exception as e:
        logging.info("Order placement failed: {}".format(e.message))
        return False
    

In [ ]:
def selectShare(sharename):
    if sharename == 'BSE:SHARE_NAME':
        share = #SHARE_NAME
    return share 

In [ ]:
#shareName, freedays, funds, maxValue, price, percent
def SelectPrices(shareName, freedays, funds, maxValue):
    share = selectShare(shareName)
    cp = share.price()  # cp = current price
    cp = float(cp)
    freedays = share.freedays
    percent = share.percent
    numShares, avgPrice = share.numSharesANDavgPrice()
    if numShares is not None:
        go = True
        lowSelect = avgPrice
    else:
        go = False     
        highSelect = r(maxValue/0.95)
    print(shareName, cp)
    print('true means sell and false means sell')
    print(go)
    if go and cp > (1+(percent/100))*lowSelect:
        print('sell', cp)
        highSelect = (1+(percent/100))*lowSelect
        quantity = numShares
        #if (t.tm_min>30 and t.tm_hour>9) and (tt.tm_hour<15 and t.tm_min<30):
        print(f'|-----------sell order placed for {shareName} for {quantity} shares at {highSelect}-----------|')
        placeOrder(shareName, quantity, False, highSelect)
    if not(go):
        #a += 1
        #print('a', a)
        if (cp<=0.95*highSelect and cp<=maxValue): #or (a>freedays):
            print('buy', cp)
            lowSelect = cp
            go = True
            quantity = int((balance()*0.1)/lowSelect)
            #if (t.tm_min>30 and t.tm_hour>9) and (tt.tm_hour<15 and t.tm_min<30):
            placeOrder(shareName, quantity, True, lowSelect)                                                     
            print(f'|-----------buy order placed for {shareName} for {quantity} shares at {lowSelect}-------------|')

In [ ]:
class Share():
    def __init__(self, shareName, tradingSymbol, exchange ,freedays, percent, funds, maxValue):
        self.shareName = shareName
        self.tradingSymbol = tradingSymbol
        self.exchange = exchange
        self.freedays = freedays
        self.percent = percent
        self.funds = funds
        self.maxValue = maxValue
    def price(self):          # ltp is lastprice
        p = kite.ltp(stock_list)
        return p[self.shareName]['last_price']
    def numSharesANDavgPrice(self):
        for i in kite.holdings():
            if i['tradingsymbol']==self.tradingSymbol and i['exchange']==self.exchange:
                print(i['tradingsymbol'], i['exchange'])
                return i['opening_quantity'], i['average_price']
        return None, None
    def PLACEORDER(self):
        SelectPrices(self.shareName, self.freedays, self.funds, self.maxValue)
        return None

In [ ]:
# name : name of stock
# tradingSymbol : symbol according to ZERODHA
# exchange : NSE or BSE
# freedays : number of days after which algorithm buys stock even if it is now below the threshold value
# after what much change in percentage we want to sell the shares
# funds : how much funds to use for particular share
# maxValue : buys If share goes below this price
# name,      tradingSymbol,                  exchange ,freedays, percent, funds, maxValue

Sharename = Share('BSE:SHARE_NAME', 'SHARE_NAME', 'EXCHANGE', 50, 10, 5000, 51) 
# (Sharename in form of code)
 

In [ ]:
iteration = 0
while True:
    iteration += 1
    Sharename.PLACEORDER()
    time.sleep(3)
    clear_output(wait=True)
    if balance()*0.1 < 100:
        break

In [ ]:
while True:
    time.sleep(3)
    if Sharename.price() < 51.5:
        placeOrder('SHARE_NAME', 10, True, Sharename.price())
        print(f'order placed at {Sharename.price()} for 10 shares')
    